In [1]:
from datasets import load_dataset
import pandas as pd
import evaluate
dataset = load_dataset("IlyaGusev/gazeta")

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 60964
    })
    test: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6793
    })
    validation: Dataset({
        features: ['text', 'summary', 'title', 'date', 'url'],
        num_rows: 6369
    })
})

In [3]:
df = pd.DataFrame(dataset['test'])
df.head()

,text,summary,title,date,url
0,На этих выходных в Берлине прошли крупные акци...,Протестующие против антикоронавирусных мер нем...,В Германии объяснили упоминание имени Путина н...,2020-09-01 00:22:59,https://www.gazeta.ru/politics/2020/08/31_a_13...
1,Высокопоставленная американская и израильская ...,"Делегации Израиля и США прилетели в ОАЭ, где о...",Делегации Израиля и США прибыли в ОАЭ для обсу...,2020-09-01 08:08:16,https://www.gazeta.ru/politics/2020/08/31_a_13...
2,Одна из руководителей Координационного совета ...,Белорусская оппозиция в лице экс-кандидата в п...,Оппозиция Белоруссии объявила о создании новой...,2020-09-01 09:21:38,https://www.gazeta.ru/politics/2020/09/01_a_13...
3,Россия считает действия ВС США во время учений...,Действия американских ВС в Эстонии во время уч...,Россия считает крайне опасными действия США на...,2020-09-01 09:33:30,https://www.gazeta.ru/army/2020/09/01/13222904...
4,С 1 сентября в России вступают в силу поправки...,Поправки в российский закон «О банкротстве» вс...,В России вступил в силу закон о внесудебном ба...,2020-09-01 09:49:24,https://www.gazeta.ru/business/2020/09/01/1322...


In [4]:
bleu = evaluate.load("bleu")

In [5]:
# ИЛИ
# from datasets import load_metric
# bleu = load_metric("sacrebleu")

In [6]:
predictions = ["Котик похож на булочку"]
references = ["Котик похож на булку"]

In [7]:
bleu.compute(predictions=predictions, references=references)

{'bleu': 0.0,
 'precisions': [0.75, 0.6666666666666666, 0.5, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 4,
 'reference_length': 4}

BLEU: "Чем ближе к человеческому переводу, тем лучше" \
Недостатки: осознанность текста не учитывается.

In [8]:
from transformers import MBartTokenizer, MBartForConditionalGeneration

model_name = "IlyaGusev/mbart_ru_sum_gazeta"
tokenizer = MBartTokenizer.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

In [9]:
import torch
device = torch.device('cuda:0')
model.to(device)

MBartForConditionalGeneration(
  (model): MBartModel(
    (shared): Embedding(250027, 1024, padding_idx=1)
    (encoder): MBartEncoder(
      (embed_tokens): Embedding(250027, 1024, padding_idx=1)
      (embed_positions): MBartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x MBartEncoderLayer(
          (self_attn): MBartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm)

In [10]:
len(dataset['test']['text'])

6793

In [11]:
from tqdm import tqdm

for text in tqdm(dataset['test']['text'][:10], total=len(dataset['test']['text'][:10])):

    input_ids = tokenizer(
        text,
        max_length=600,
        padding="max_length",
        truncation=True,
        return_tensors="pt",
    )["input_ids"]
    input_ids = input_ids.to(device)
    output_ids = model.generate(
        input_ids=input_ids,
        no_repeat_ngram_size=4
    )[0]

    summary = tokenizer.decode(output_ids, skip_special_tokens=True)
    # задача -- для каждого текста посчитать bleu, и вывести среднюю метрику по датасету dataset['test']

100%|██████████| 10/10 [00:18<00:00,  1.89s/it]


In [12]:
print(summary)

Президент США Дональд Трамп назвал «марионеткой» своего соперника на предстоящих выборах демократа Джо Байдена. Если Байден придет к власти, то его быстро отберут радикальные левые силы, которые устроят в Америке революцию, заявил Трамп. Он также подчеркнул, что не желает новой гражданской войны в США — между «леваками» и его сторонниками. Накануне Байден обвинил Трампа в «покорности» перед российскими властями.


In [13]:
for i in range(5):
    print(tokenizer.decode(output_ids[i + 1]) + '\n')

Президент

США

Дональд

Трамп

назвал



In [14]:
prediction = list([summary])
reference = [text]

In [15]:
prediction

['Президент США Дональд Трамп назвал «марионеткой» своего соперника на предстоящих выборах демократа Джо Байдена. Если Байден придет к власти, то его быстро отберут радикальные левые силы, которые устроят в Америке революцию, заявил Трамп. Он также подчеркнул, что не желает новой гражданской войны в США — между «леваками» и его сторонниками. Накануне Байден обвинил Трампа в «покорности» перед российскими властями.']

In [16]:
reference

['Президент США Дональд Трамп назвал «марионеткой» своего соперника на предстоящих выборах американского лидера, демократа Джо Байдена. Об этом сообщает телеканал Fox News. Если Байден придет к власти, то ее очень быстро отберут радикальные левые силы, которые устроят в Америке революцию, заявил Трамп. «Байден… он слабый человек и был таким всю свою жизнь. Его контролируют, как куклу... это революция. Он не должен баллотироваться в президенты», — отметил он. В ответ на вопрос телеведущей, кто же управляет «марионеткой Байденом», Трамп заявил, что она «этих людей не знает» и вдаваться в подробности не стал. Он также подчеркнул, что не желает новой гражданской войны в США — между «леваками» и его сторонниками. «Этого я не хочу, я оставляю это [столкновения с радикалами] нашим правоохранителям», — добавил Трамп. Накануне Джо Байден обвинил Трампа в «покорности» перед российскими властями. «Мы видели сообщения, что российские силы атаковали американских солдат в Сирии, поранили наших солда

In [17]:
bleu.compute(predictions=prediction, references=reference)

{'bleu': 0.00013714348069350797,
 'precisions': [0.9850746268656716,
  0.9393939393939394,
  0.8615384615384616,
  0.78125],
 'brevity_penalty': 0.00015437599535098924,
 'length_ratio': 0.10229007633587786,
 'translation_length': 67,
 'reference_length': 655}